# Лабораторна робота №2
**Тема:** РЕГРЕСІЯ

## Мета роботи
Закріпити основні поняття використання моделі лінійної регресії в машинному навчанні

Завдання:
•	Прочитайте файл .csv у DataFrame pandas.
•	Досліджуйте набір даних за допомогою бібліотек візуалізації Python.
•	Експериментуйте з різними ознаками для створення моделі лінійної регресії.
•	Налаштуйте гіперпараметри моделі.
•	Порівняйте результати навчання, використовуючи середню квадратичну помилку та графіки втрат.
•	Спробуйте побудувати модель та використати регресії для інших ознак та міток.



**Хід роботи**

Для виконання завдання будемо використовувати файл з даними, який доступный за посиланням: [City of Chicago Taxi Trips dataset](https://data.cityofchicago.org/Transportation/Taxi-Trips-2013-2023-/wrvz-psew/about_data)
В ньому розташовані дані про таксі Чикаго.

Перегляньте, яка інформація там є https://data.cityofchicago.org/Transportation/Taxi-Trips-2024-/ajtu-isnz/about_data


**Крок 1.** Імпортуйте необхідні бібліотеки

In [ ]:
# Робота з даними
import pandas as pd
import numpy as np

# Візуалізація
import matplotlib.pyplot as plt
import seaborn as sns

# Моделювання
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


**Крок 2**. Завантажте дані з хмари

In [ ]:
url = "https://data.cityofchicago.org/resource/wrvz-psew.csv?$limit=1000"  # приклад: беремо 1000 рядків
df = pd.read_csv(url)


print(df.shape)
df.head()

**Крок 3.** Вибір даних потрібних для навчання

Для побудови моделі лінійної регресії нам не потрібні всі поля з набору таксі Чикаго. Ми оберемо лише ті, що мають сенс як ознаки (features) та ціль (target)

Основні колонки для моделі

trip_start_timestamp – час початку поїздки (для ознак: година, день тижня, вихідний).

trip_end_timestamp – час завершення (для тривалості).

trip_miles – відстань поїздки.

fare – тариф (може бути цільовою змінною).

tips – чайові (альтернативна ціль).

payment_type – тип оплати (категоріальна ознака).

pickup_community_area – район посадки (категоріальна ознака).

dropoff_community_area – район висадки (категоріальна ознака).

Формування нового DataFrame:

In [ ]:
selected_cols = [
    "trip_start_timestamp",
    "trip_end_timestamp",
    "trip_miles",
    "fare",
    "tips",
    "payment_type",
    "pickup_community_area",
    "dropoff_community_area"
]

df_model = df[selected_cols].copy()

print(df_model.shape)
df_model.head()


**Крок 4.** Очищення та підготовка даних

Основні кроки очищення:

Перевірка пропусків. Виявити колонки з великою кількістю NaN.Вирішити: видалити рядки, заповнити середнім/медіаною, або залишити як є.

Видалення дублікатів. Перевірити, чи є повторні записи поїздок.

Фільтрація аномалій. Відстань (trip_miles) не може бути від’ємною чи надто великою.Тривалість (trip_duration_min) має бути > 0 і реалістичною.Тариф (fare) ≥ 0.

Перетворення типів даних. Дати → datetime. Категорії → category. Числові поля → float або int.

In [ ]:
# 1. Перетворення дат
df_model['trip_start_timestamp'] = pd.to_datetime(df_model['trip_start_timestamp'], errors='coerce')
df_model['trip_end_timestamp'] = pd.to_datetime(df_model['trip_end_timestamp'], errors='coerce')

# 2. Створення тривалості поїздки
df_model['trip_duration_min'] = (
    df_model['trip_end_timestamp'] - df_model['trip_start_timestamp']
).dt.total_seconds() / 60

# 3. Видалення пропусків у ключових колонках
df_model = df_model.dropna(subset=['trip_start_timestamp','trip_end_timestamp','trip_miles','fare'])

# 4. Видалення дублікатів
df_model = df_model.drop_duplicates()

# 5. Фільтрація аномалій
df_model = df_model[(df_model['trip_miles'] > 0) & (df_model['trip_miles'] < 100)]
df_model = df_model[(df_model['trip_duration_min'] > 0) & (df_model['trip_duration_min'] < 300)]
df_model = df_model[df_model['fare'] >= 0]

# 6. Категоріальні змінні
df_model['payment_type'] = df_model['payment_type'].astype('category')
df_model['pickup_community_area'] = df_model['pickup_community_area'].astype('category')
df_model['dropoff_community_area'] = df_model['dropoff_community_area'].astype('category')

print(df_model.info())
print(df_model.head())

**Крок 5.** Основні статистичні показники

Для числових колонок (trip_miles, fare, tips, trip_duration_min) ми розрахуємо:

count – кількість спостережень

mean – середнє значення

std – стандартне відхилення

min / max – мінімум та максимум

25%, 50%, 75% - – квартилі (медіана та інтервали)

In [ ]:
# Основні статистики для числових колонок
stats = df_model[['trip_miles','fare','tips','trip_duration_min']].describe()

print(stats)

# Додатково: коефіцієнт кореляції між ознаками
corr = df_model[['trip_miles','fare','tips','trip_duration_min']].corr()

print("\nКореляція між числовими ознаками:")
print(corr)

**Інтерпретація:**

Середня відстань поїздки ≈ 3.2 милі, більшість поїздок короткі.

Середній тариф ≈ $12–13, але є поїздки з тарифом понад $80.

Чайові мають сильний перекіс: медіана = $1, але максимум може бути $15+.

Тривалість поїздки ≈ 18 хвилин, більшість у діапазоні 10–25 хв.

Кореляція: тариф сильно корелює з відстанню та тривалістю, чайові — слабше.

**Крок 6.** Візуалізуємо дані кореляції у вигляді графіку

In [ ]:
# Стиль графіків
plt.style.use('seaborn-v0_8')

# 1. Гістограма тарифів
plt.figure(figsize=(6,4))
sns.histplot(df_model['fare'], bins=50, kde=True)
plt.title("Розподіл тарифів (fare)")
plt.xlabel("Fare ($)")
plt.ylabel("Кількість поїздок")
plt.show()

# 2. Гістограма відстаней
plt.figure(figsize=(6,4))
sns.histplot(df_model['trip_miles'], bins=50, kde=True)
plt.title("Розподіл відстаней (trip_miles)")
plt.xlabel("Miles")
plt.ylabel("Кількість поїздок")
plt.show()

# 3. Розсіювання між відстанню та тарифом
plt.figure(figsize=(6,4))
sns.scatterplot(x='trip_miles', y='fare', data=df_model.sample(500, random_state=42), alpha=0.3)
plt.title("Залежність тарифу від відстані")
plt.xlabel("Miles")
plt.ylabel("Fare ($)")
plt.show()

# 4. Кореляційна матриця
plt.figure(figsize=(6,5))
corr = df_model[['trip_miles','fare','tips','trip_duration_min']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Кореляція між числовими ознаками")
plt.show()


**Інтерпретація:**

Гістограми: більшість поїздок короткі (2–4 милі) з тарифами $10–15.

Scatter plot: чітка лінійна залежність між милями та тарифом, але є відхилення (наприклад, довгі поїздки з низьким тарифом).

Heatmap: тариф сильно корелює з відстанню та тривалістю, чайові мають слабший зв’язок.

**Крок 7.**

Побудуємо регресійну модель, яка прогнозує залежність вартості поїздки від її тривалості.

In [ ]:
# Ознака: тривалість поїздки
X = df_model[['trip_duration_min']]
# Ціль: тариф
y = df_model['fare']

# Видаляємо пропуски
data = pd.concat([X, y], axis=1).dropna()
X_clean = data[['trip_duration_min']]
y_clean = data['fare']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_clean, test_size=0.2, random_state=42)

# Лінійна регресія
lr = LinearRegression()
lr.fit(X_train, y_train)

# Прогноз та оцінка
y_pred = lr.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("Intercept (базова вартість):", lr.intercept_)
print("Coefficient (зміна тарифу за 1 хв):", lr.coef_[0])

print("MSE на тестових даних:", mse)

# Візуалізація: тривалість vs тариф
plt.figure(figsize=(6,6))
plt.scatter(X_test[:300], y_test[:300], alpha=0.3, label="Реальні дані")
plt.scatter(X_test[:300], y_pred[:300], alpha=0.3, label="Прогноз")
plt.xlabel("Тривалість поїздки (хв)")
plt.ylabel("Fare ($)")
plt.title("Прогноз тарифу від тривалості")
plt.legend()
plt.show()

**Інтерпретація:**

Залежність між тривалістю та тарифом

Модель показує, що тариф має позитивну лінійну залежність від тривалості поїздки.Чим довше триває поїздка, тим вищою є її вартість. Це логічно, адже довші поїздки зазвичай означають більшу відстань і більший час роботи водія.

Коефіцієнт регресії. Значення коефіцієнта при trip_duration_min показує, на скільки доларів у середньому змінюється тариф при збільшенні тривалості на 1 хвилину. В нашому випадку  коефіцієнт = 0.72, то кожна додаткова хвилина додає приблизно $0.72 до вартості.

Середня квадратична помилка (MSE) показує, наскільки прогноз відхиляється від реальних значень.MSE (Mean Squared Error) вимірює середній квадрат різниці між реальними та прогнозованими значеннями.Чим менше MSE, тим ближче модель до реальних даних. Якщо ми прогнозували вартість поїздки (fare) від тривалості (trip_duration_min), то MSE ≈ 148 означає, що середня похибка моделі у квадраті складає ~148. Це досить велике значення, враховуючи, що середні тарифи у наборі даних — близько $10–15.Тобто модель на основі лише тривалості не дуже точна: вона пропускає інші важливі фактори (відстань, базова ставка, район, тип оплати).





**Крок 8.** Використаємо побудовану модель для прогнозування вартості поїздки, якщо відома її тривалість.

In [ ]:
# Приклад нових даних
new_data = pd.DataFrame({
    'trip_duration_min': [15, 30]

})

predicted_fares = lr.predict(new_data)
print(predicted_fares)

[16.80548958 27.74065139]


**Завдання для самостійного виконання**

Створіть регресійну модель, яка додає до параметру тривалість поїздки інши параметри. Оцініть, чи покращується якість моделі.

1. Виберіть цільову змінну

Використайте вартість поїздки (fare) як ціль.

2. Сформуйте набір ознак

Обов’язково включіть тривалість поїздки (trip_duration_min).

3. Додайте інші параметри, наприклад:

trip_miles (відстань поїздки),

hour (година доби),

day_of_week (день тижня),

payment_type (тип оплати, через one-hot кодування).

4. Розділіть дані на train/test

Використайте train_test_split (наприклад, 80% для навчання, 20% для тесту).

5. Побудуйте модель лінійної регресії

6. Навчіть модель на тренувальних даних.

7. Зробіть прогноз на тестових даних.

8. Оцініть якість моделі

Обчисліть MSE (середню квадратичну помилку).

Порівняйте результат із моделлю, яка використовувала лише trip_duration_min.

9. висновок

Чи зменшилася похибка?

Які параметри найбільше вплинули на тариф?



In [ ]:
# Тут має бути Ваш код



Тут мають бути Ваші висновки

**Крок 8. Поліноміальна регресія (цикл по ступенях)**

Імпортуємо необхідні бібліотеки:

PolynomialFeatures - Трансформер ознак.Створює нові ознаки, які є степенями та добутками існуючих ознак. Наприклад, якщо у вас є ознака x, цей клас може створити x^2, x^3 і т.д. Це дозволяє лінійній моделі (LinearRegression) вивчати нелінійні залежності.

make_pipeline	 - Функція створення конвеєра.	Об'єднує декілька кроків (наприклад, PolynomialFeatures → LinearRegression) в одну модель. Це спрощує код і запобігає помилкам, гарантуючи, що до тестових даних будуть застосовані ті ж самі перетворення, що й до навчальних.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score



# 2. Очищення та відбір даних
df_clean = df[['trip_miles', 'trip_total']].dropna()

# Фільтруємо нереалістичні нульові або від'ємні значення,
# а також обмежуємо поїздки до 40 миль для кращої візуалізації
df_clean = df_clean[
    (df_clean['trip_miles'] > 0.1) &
    (df_clean['trip_total'] > 1.0) &
    (df_clean['trip_miles'] < 40)
]

print(f"Розмір очищеного датасету: {df_clean.shape}")

Розмір очищеного датасету: (840, 2)


Знаходимо залежність між двома ознаками через побудову поліному 3 ступеню

In [ ]:
# Ознака (X): Пройдена відстань
X = df_clean[['trip_miles']].values

# Цільова змінна (y): Загальна вартість
y = df_clean['trip_total'].values

# Створюємо послідовність точок для побудови гладкої кривої регресії
X_seq = np.linspace(X.min(), X.max(), 300).reshape(-1, 1)
# Встановлюємо ступінь полінома
P_DEGREE = 10

# Створюємо конвеєр (Pipeline):
# Спочатку: PolynomialFeatures(3) — створює ознаки x, x^2, x^3
# Потім: LinearRegression() — навчається на цих нових ознаках
model_poly = make_pipeline(PolynomialFeatures(degree=P_DEGREE), LinearRegression())

# Навчання моделі на даних
model_poly.fit(X, y)

# Прогноз для всіх точок, щоб отримати гладку лінію для графіка
y_pred_poly = model_poly.predict(X_seq)

# Оцінка якості моделі на початкових даних
y_fit_all = model_poly.predict(X)
r2 = r2_score(y, y_fit_all)
mse = mean_squared_error(y, y_fit_all)

print(f"\nРезультати для полінома {P_DEGREE}-го ступеня:")
print(f"R-квадрат (R2): {r2:.4f}")
print(f"Середня квадратична помилка (MSE): {mse:.2f}")

# Візуалізація
plt.figure(figsize=(10, 6))

# Точки даних
plt.scatter(X, y, color='blue', alpha=0.5, s=20, label='Фактичні дані (Trip Miles vs Trip Total)')

# Крива регресії
plt.plot(X_seq, y_pred_poly, color='red', linewidth=3, label=f'Поліноміальна регресія (ступінь {P_DEGREE})')

plt.title(f'Поліноміальна регресія ({P_DEGREE} ст.): Прогноз вартості таксі')
plt.xlabel('Відстань поїздки (trip_miles)')
plt.ylabel('Загальна вартість (trip_total, $)')
plt.legend()
plt.grid(True)
plt.ylim(0, 150) # Обмежимо вісь Y для кращої читабельності
plt.show()

Ваші висновки: (Чи краще модель поліному 3 ступеню описує залежності ніж лінійна модель?)

**Крок 9** Переробіть цей код, щоб побудувати модель високої складності, наприклад P_DEGREE=10. Зробіть висновки (чи спостерігається перенавчання?)

In [ ]:
#Ваш код тут:


**Ваші висновки тут**

**Крок 10** Проаналізуємо, як регуляризація бореться з перенавчанням

Регуляризація — це метод, який додає до функції втрат (наприклад, MSE) штраф за велику складність моделі.

 У регресії складність моделі вимірюється абсолютними значеннями її коефіцієнтів (w_i).

 Гребенева регресія (Ridge Regression, L2)- Додає до функції втрат суму квадратів усіх коефіцієнтів, помножену на коефіцієнт регуляризації альфа.

 Лассо-регресія (Lasso Regression, L1) - Додає до функції втрат суму абсолютних значень усіх коефіцієнтів, помножену на коефіцієнт регуляризації альфа.

 Це підтверджує, що регуляризація успішно знизила помилку на нових даних, а аналіз коефіцієнтів Lasso покаже, які саме високі степені полінома були визнані непотрібними і були обнулені.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler

# !!! ПРИПУСКАЄМО, ЩО X та y ВЖЕ ВИЗНАЧЕНІ (ОЧИЩЕНІ ДАНІ ТАКСІ) !!!

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- Налаштування ---
P_DEGREE = 15
ALPHA_VALUE = 0.5
print(f"--- Результати для полінома {P_DEGREE}-го ступеня (alpha={ALPHA_VALUE}) ---")

# Базова модель (без регуляризації) для порівняння перенавчання
model_base = make_pipeline(
    PolynomialFeatures(degree=P_DEGREE),
    StandardScaler(),
    LinearRegression() # Використовуємо звичайну лінійну регресію
)

# Модель L2 (Ridge)
model_ridge = make_pipeline(
    PolynomialFeatures(degree=P_DEGREE),
    StandardScaler(),
    Ridge(alpha=ALPHA_VALUE)
)

# Модель L1 (Lasso)
model_lasso = make_pipeline(
    PolynomialFeatures(degree=P_DEGREE),
    StandardScaler(),
    Lasso(alpha=ALPHA_VALUE, max_iter=20000)
)

# Навчання на train та оцінка на test
model_base.fit(X_train, y_train)
model_ridge.fit(X_train, y_train)
model_lasso.fit(X_train, y_train)

# Оцінка MSE на ТЕСТОВІЙ ВИБІРЦІ
mse_base_test = mean_squared_error(y_test, model_base.predict(X_test))
mse_ridge_test = mean_squared_error(y_test, model_ridge.predict(X_test))
mse_lasso_test = mean_squared_error(y_test, model_lasso.predict(X_test))

print(f"Базова MSE (ТЕСТ): {mse_base_test:.2f} (Показує перенавчання)")
print(f"✅ Ridge MSE (ТЕСТ): {mse_ridge_test:.2f} (L2 - зменшує коефіцієнти)")
print(f"✅ Lasso MSE (ТЕСТ): {mse_lasso_test:.2f} (L1 - обнуляє коефіцієнти)")


# === ФІКС: Коефіцієнти потрібно брати з об'єкта всередині конвеєра ===
lasso_model_step = model_lasso.named_steps['lasso']

# Коефіцієнти (w1, w2, ..., w15)
weights = lasso_model_step.coef_

# Зміщення (w0)
intercept = lasso_model_step.intercept_
# ===================================================================

# Скільки коефіцієнтів стали фактично нульовими (дуже близькими до 0)
zero_count = np.sum(np.abs(weights) < 1e-4)
non_zero_count = len(weights) - zero_count

print("\n--- Аналіз коефіцієнтів Lasso (L1) ---")
print(f"Вільний член (w0): {intercept:.4f}")
print(f"Кількість навчених коефіцієнтів (w1...w15): {len(weights)}")
print(f"Кількість ненульових коефіцієнтів: {non_zero_count} (ознаки, що залишилися)")
print(f"Кількість обнулених коефіцієнтів: {zero_count} (ознаки, виключені L1)")

print("\nТоп-5 найбільших коефіцієнтів:")
degrees = np.arange(1, len(weights) + 1)
sorted_indices = np.argsort(np.abs(weights))[::-1]

for i in sorted_indices[:5]:
    print(f"W_{degrees[i]} (для x^{degrees[i]}): {weights[i]:.4f}")